In [2]:
import pandas as pd
import sys
sys.path.append("/Users/georgiikuznetsov/Desktop/rl_rebalance")
from src.targets import *

In [ ]:
from src.baselines import *

returns = pd.read_parquet("../data/returns.parquet")
targets = pd.read_parquet("../data/targets_minvar.parquet")

returns, targets = returns.align(targets, join = "inner")

tickers = list(returns.columns)
N = len(tickers)

weight_low = np.zeros(N, dtype = float)
weight_high = np.ones(N, dtype = float) * 0.8
leverage_cap = 1.0
daily_turnover_cap = 0.03

tc_c = np.array([1.5] * N) / 1.0   # linear term in bps per 100% weight change
tc_k = np.array([5.0] * N) / 1.0   # impact factor (tune later)

In [4]:
from src.eval import metrics_from_pnl

baseline_results = {}

# === PERIODIC ===
res_monthly = simulate_periodic(
    returns=returns,
    targets=targets,
    rebalance_freq="M",
    weight_low=weight_low,
    weight_high=weight_high,
    leverage_cap=leverage_cap,
    daily_turnover_cap=daily_turnover_cap,
    tc_c=tc_c,
    tc_k=tc_k
)
baseline_results["periodic_monthly"] = metrics_from_pnl(
    res_monthly["return"].values,
    tc_bps=res_monthly["tc"].values * 1e4
)

# === BAND ===
for band in [0.02, 0.04]:
    res_band = simulate_band(
        returns=returns,
        targets=targets,
        band=band,
        weight_low=weight_low,
        weight_high=weight_high,
        leverage_cap=leverage_cap,
        daily_turnover_cap=daily_turnover_cap,
        tc_c=tc_c,
        tc_k=tc_k
    )
    baseline_results[f"band_{int(band*100)}bps"] = metrics_from_pnl(
        res_band["return"].values,
        tc_bps=res_band["tc"].values * 1e4
    )

C:\Users/zih028/Documents/GitHub/rl_rebalance\src\baselines.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebal_dates = pd.Series(1, index=dates).resample(rebalance_freq).first().index


In [6]:
baseline_table = pd.DataFrame(baseline_results).T[
    ["ann_ret", "ann_vol", "sharpe", "max_dd", "avg_tc_bps"]
]

print(baseline_table.round(4))


                  ann_ret  ann_vol  sharpe  max_dd  avg_tc_bps
periodic_monthly   0.0730   0.1620  0.4508 -0.4730      0.0020
band_2bps          0.0842   0.1701  0.4952 -0.5175      0.0020
band_4bps          0.0804   0.1613  0.4985 -0.4799      0.0011
